In [14]:
import pandas as pd
import numpy as np

import numpy as np
import matplotlib.pyplot as plt

import nitime
import nitime.timeseries as ts
import nitime.viz as viz

In [20]:
coord = pd.read_csv('dat\coordinates.txt', sep = ' ', index_col='Index')

coord_arr = np.array(coord[['X', 'Y', 'Z']].apply(lambda x: x.replace('−', '-')))

FileNotFoundError: [Errno 2] No such file or directory: 'dat\\coordinates.txt'

In [ ]:
coord_arr = [list(map(lambda x: int(x.replace('−', '-')), y)) for y in coord_arr]


In [ ]:
adj_mat = np.random.uniform(size = (116,116))

In [ ]:
from nilearn import plotting
plotting.plot_connectome(adj_mat, node_coords=coord_arr)

In [ ]:

TR = 2.

In [16]:
def get_data(entry):
    variables = entry[['var1', 'var2', 'var3']].replace({"A": 0, "C": 1}).to_numpy()
    timeseries = entry[5 if "y" in entry.index else 4:].to_numpy(dtype=float).reshape((115, 116), order="F")
    return {"variables": variables, "timeseries": timeseries}


In [17]:
data = pd.read_csv('./data/train.csv')
t1 = ts.TimeSeries(get_data(data.iloc[0])["timeseries"], sampling_interval=TR)

NameError: name 'TR' is not defined

In [ ]:
viz.plot_tseries(t1)
